example https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3539694

In [95]:
#!if [ ! -f ArticleScraper.ipynb ]; then 
!curl https://raw.githubusercontent.com/Karocyt/covid-scraping/master/ArticleScraper.ipynb > ArticleScraper.ipynb
#; else echo "Already done"; fi
%run ArticleScraper.ipynb

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17261  100 17261    0     0  87176      0 --:--:-- --:--:-- --:--:-- 87619
Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/marut

AttributeError: ignored

In [96]:
!pip3 install PyPDF2

from selenium.webdriver.common.by import By
import time
import io
import PyPDF2

In [0]:
example_dicts = [{'authors': 'C You, Y Deng, W Hu, J Sun, Q Lin, F Zhou… - Available at SSRN …, 2020 - papers.ssrn.com',
  'extra_link': 'https://www.medrxiv.org/content/medrxiv/early/2020/02/11/2020.02.08.20021253.full.pdf',
  'extra_link_text': '[PDF] medrxiv.org',
  'link': 'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3539694',
  'preview': 'Background: The 2019 novel coronavirus (2019-nCoV) outbreak in Wuhan, China has\nattracted world-wide attention. As of February 11, 2020, a total of 44,730 cases of\npneumonia associated with the 2019-nCoV were confirmed by the National Health …',
  'title': 'Estimation of the Time-Varying Reproduction Number of COVID-19 Outbreak in China'},
 {'authors': 'J Gao, P Zheng, Y Jia, H Chen, Y Mao… - Available at SSRN …, 2020 - papers.ssrn.com',
  'extra_link': '',
  'extra_link_text': '',
  'link': 'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3541120',
  'preview': 'Background: Huge citizens expos social media during a novel coronavirus disease (COVID-\n19) outbroke in Wuhan, China. We assess the prevalence of mental health problems and\nexamine their association with social media exposure. Methods: We conducted a cross …',
  'title': 'Mental Health Problems and Social Media Exposure During COVID-19 Outbreak'},
 {'authors': 'C GU, W Jiang, T Zhao, B Zheng - Available at SSRN 3551006, 2020 - papers.ssrn.com',
  'extra_link': '',
  'extra_link_text': '',
  'link': 'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3551006',
  'preview': 'The statistics show that the mortality of COVID-19 is 20 times higher than seasonal flu and\nclose to that of Spanish flu, hence it is becoming an absolute priority for every country to take\nefficient measure to limit the transmission of COVID-19. In this short paper, we propose a …',
  'title': 'Mathematical Recommendations to Fight Against COVID-19'},
  ]

In [0]:
class SsrnSingleScraper(ArticleScraper):
  page_dict = None

  def __init__(self, page_dict):
    ArticleScraper.wd.get(page_dict['link'])
    self.page_dict = page_dict
    ArticleScraper.__init__(self, page_dict['link'])

  @property
  def title(self):
    return self.page.head.title.text.split(' by ')[0]

  @property
  def authors(self):
    return self.page.head.title.text.split(' by ')[1].split(" ::")[0]

  @property
  def doi(self):
    return 'SSRN-ID' + self.page_dict['link'].split('=')[1]

  @property
  def abstract(self):
    div = self.page.find("div", {"class": "abstract-text"})
    text = div.find("p").text
    return text

  @property
  def date(self):
    xpath = '//*[@id="selectable"]/text()'
    text = self.tree.xpath(xpath)[0]
    try:
      date = datetime.strptime(text.split(")")[-2].split("(")[-1], '%m/%d/%Y')
    except:
      date = datetime.strptime(text.split(")")[-2].split("(")[-1], '%B %d, %Y')
    return date.strftime(self.date_format)

  @property
  def body(self):
    time.sleep(0.5)
    ArticleScraper.wd.find_element(By.XPATH, '//*[@id="maincontent"]/div[1]/div[1]/div/div[2]/a').click()
    time.sleep(0.5)
    content = ""
    print(os.listdir()) ### Not downloaded anymore ?! Despite waits everywhere ?! Time to sleep
    for file in os.listdir():
      if file.endswith(".pdf"):
        with open(file, 'rb') as f:
          pdf_reader = PyPDF2.PdfFileReader(f)
          for page in pdf_reader.pages:
            content += page.extractText()
        os.remove(file)
        break
    
    print(len(content)) ### TO CLEAN (raw)
    return None

  @property
  def source(self):
    return "SSRN"

  @property
  def source_impact_factor(self):
    return None

  @property
  def search_keyword(self):
    return None

  @property
  def categories(self):
    return None

  @property
  def licence(self):
    return None

  @property
  def citations(self):
    return None

  @property
  def organization(self):
    # Using organization of lead author
    try:
      authors = ArticleScraper.wd.find_element(By.CLASS_NAME, 'authors-full-width') ##### TO FIX
      return authors.find_element(By.TAG_NAME, 'p').text
    except Exception as e:
      print("ERROR handling org name:", e)
    return None

  @property
  def keywords(self):
    keywords = self.page.find(text='Keywords:').parent.parent.text.split(': ')[1]
    return ",".join(keywords.split(';'))

  @property
  def references(self):
    wd = ArticleScraper.wd
    wd.find_element_by_xpath('//*[@id="references-widget"]/button').click() # Not working anymore ?! WTF ? Need screenshot and sleep
    time.sleep(0.2)
    li_list = wd.find_elements_by_xpath('//*[@id="references-widget"]/ol/li')
    refs = []
    for elem in li_list:
      ref = {}
      try:
        ref['title'] = elem.find_element(By.CLASS_NAME, 'reference-title').text
      except:
        ref['title'] = ""
      
      try:
        ref['authors'] = elem.find_element(By.CLASS_NAME, 'author-list').text
      except:
        ref['authors'] = ""
      
      try:
        link = elem.find_element(By.TAG_NAME,  'a')
        ref['link'] = link.get_attribute('href')
      except:
        ref['link'] = ""
      refs.append(ref)
    
    return refs

  @property
  def link(self):
    return self.page_dict['link']

  @property
  def extralinks(self):
    return self.page_dict['extra_link']
  

In [120]:
class SsrnScraper():
  def parse(self, pages_dict):
    for d in pages_dict:
      yield SsrnSingleScraper(d).parse()  

scraper = SsrnScraper()
for d in scraper.parse(example_dicts):
  for k in d:
    print(k, d[k], sep=": ")
  print()

['.config', 'ArticleScraper.ipynb', 'covid-scraping', 'ArticleScraper.ipyn', 'sample_data']
0
Title: Estimation of the Time-Varying Reproduction Number of COVID-19 Outbreak in China
Authors: Chong You, Yuhao Deng, Wenjie Hu, Jiarui Sun, Qiushi Lin, Feng Zhou, Cheng Heng Pang, Yuan Zhang, Zhengchao Chen, Xiao-Hua Zhou
DOI: SSRN-ID3539694
Abstract: Background: The 2019 novel coronavirus (2019-nCoV) outbreak in Wuhan, China has attracted world-wide attention. As of February 11, 2020, a total of 44,730 cases of pneumonia associated with the 2019-nCoV were confirmed by the National Health Commission (NHC) of China.    Methods: Three approaches, namely Poisson likelihood-based method (ML), exponential growth rate-based method (EGR) and stochastic Susceptible-Infected-Removed dynamic model-based method (SIR), were implemented to estimate the basic and controlled reproduction numbers.    Results: A total of 71 chains of transmission together with dates of symptoms onset and 67 dates of infecti

NoSuchElementException: ignored